In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# --- Step 1: Fetch daily PM2.5, temperature, humidity for London from AQICN ---
aqicn_token = "89b21bfd45ccd18a7868916d4acd03b8cd2d97db"
base_url = "https://api.waqi.info/feed/london/?token=" + aqicn_token

# Use timezone-aware datetime for future compatibility
now = datetime.now().astimezone().replace(tzinfo=None)
dates = [(now - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(6, -1, -1)]
pm25_list, temp_list, hum_list = [], [], []

for date in dates:
    # AQICN only provides current data, so we fetch the latest for each day (not historical hourly)
    resp = requests.get(base_url)
    data = resp.json()
    if data.get("status") == "ok":
        iaqi = data["data"]["iaqi"]
        pm25 = iaqi.get("pm25", {}).get("v", None)
        temp = iaqi.get("t", {}).get("v", None)
        hum = iaqi.get("h", {}).get("v", None)
    else:
        pm25 = temp = hum = None
    pm25_list.append(pm25)
    temp_list.append(temp)
    hum_list.append(hum)

# --- Step 2: Fetch daily global CO2 from Mauna Loa (NOAA) ---
url = "https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_trend_gl.txt"
resp = requests.get(url)
lines = resp.text.splitlines()
data_lines = [line for line in lines if not line.startswith("#") and line.strip()]
# Get last 7 days
co2_list = []
co2_dates = []
for line in data_lines[-7:]:
    year, month, day, co2, *_ = line.split()
    co2_dates.append(f"{year}-{month.zfill(2)}-{day.zfill(2)}")
    co2_list.append(float(co2))

# --- Step 3: Combine into DataFrame ---
df = pd.DataFrame({
    "Date": dates,
    "PM2.5 (µg/m³)": pm25_list,
    "Temperature (°C)": temp_list,
    "Humidity (%)": hum_list,
    "CO₂ (ppm)": co2_list
})
df["Date"] = pd.to_datetime(df["Date"])

print("Daily air quality and CO₂ readings for London and globally (last 7 days):")
print(df)

# --- Step 4: Plot all variables ---
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.plot(df["Date"], df["PM2.5 (µg/m³)"], color="red", marker="o", label="PM2.5 (µg/m³)")
ax1.set_ylabel("PM2.5 (µg/m³)", color="red")
ax2 = ax1.twinx()
ax2.plot(df["Date"], df["Temperature (°C)"], color="blue", marker="x", label="Temperature (°C)")
ax2.plot(df["Date"], df["Humidity (%)"], color="green", marker="s", label="Humidity (%)")
ax2.plot(df["Date"], df["CO₂ (ppm)"], color="black", marker="*", label="CO₂ (ppm)")
ax2.set_ylabel("Temperature (°C) / Humidity (%) / CO₂ (ppm)")
fig.legend(loc="upper right")
plt.title("London PM2.5, Temperature, Humidity, and Global CO₂ (Last 7 Days)")
plt.tight_layout()
plt.show()

print("\nHow to read this chart:")
print("- Red line: PM2.5 (fine particulate matter, micrograms per cubic meter) for London. Lower is better for health.")
print("- Blue line: Temperature in degrees Celsius for London.")
print("- Green line: Relative humidity in percent for London.")
print("- Black line: Global daily CO₂ (parts per million) from Mauna Loa Observatory.")
print("All data is live and up to date. You can use this chart for your report or demo.")
